In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import normalized_mutual_info_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy.optimize import linear_sum_assignment
from gower_duped import gower_matrix as gower_matrix_duped

In [2]:
def cluster_accuracy(y_pred, y_true):
    # We need to map the labels to our cluster labels
    # This is a linear assignment problem on a bipartite graph
    k = max(len(np.unique(y_pred)), len(np.unique(y_pred)))
    cost_matrix = np.zeros((k, k))
    for i in range(y_pred.size):
        cost_matrix[y_pred[i], y_true[i]] += 1
    row_ind, col_ind = linear_sum_assignment(cost_matrix.max() - cost_matrix)
    return cost_matrix[row_ind, col_ind].sum() / y_pred.size

In [3]:
og_df = pd.read_csv("datasets/census_income.csv")
og_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K.
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


In [4]:
og_df.loc[(og_df["class"] == " <=50K.") | (og_df["class"] == " <=50K"), "class"] = 0
og_df.loc[(og_df["class"] == " >50K.") | (og_df["class"] == " >50K"), "class"] = 1
# Probability of most common class
og_df["class"].value_counts().max()/og_df["class"].count()

0.7607182343065395

In [5]:
cat_cols = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
cont_cols = ["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]

In [6]:
df = og_df.copy()
df.drop(columns="class", inplace=True)
df[cat_cols] = df[cat_cols].apply(LabelEncoder().fit_transform)
df[cont_cols] = MinMaxScaler().fit_transform(df[cont_cols])
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,0.301370,7,0.044131,9,0.800000,4,1,1,4,1,0.021740,0.0,0.397959,39
1,0.452055,6,0.048052,9,0.800000,2,4,0,4,1,0.000000,0.0,0.122449,39
2,0.287671,4,0.137581,11,0.533333,0,6,1,4,1,0.000000,0.0,0.397959,39
3,0.493151,4,0.150486,1,0.400000,2,6,0,2,1,0.000000,0.0,0.397959,39
4,0.150685,4,0.220635,9,0.800000,2,10,5,2,0,0.000000,0.0,0.397959,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.301370,4,0.137428,9,0.800000,0,10,1,4,0,0.000000,0.0,0.357143,39
48838,0.643836,0,0.209130,11,0.533333,6,0,2,2,1,0.000000,0.0,0.397959,39
48839,0.287671,4,0.245379,9,0.800000,2,10,0,4,1,0.000000,0.0,0.500000,39
48840,0.369863,4,0.048444,9,0.800000,0,1,3,1,1,0.054551,0.0,0.397959,39


In [7]:
def encode_feature(df, feature_to_encode):
    dummies = pd.get_dummies(df[[feature_to_encode]], dtype=float)
    result_df = pd.concat([df, dummies], axis=1)
    result_df.drop(columns=feature_to_encode, inplace=True)
    return result_df

df_one_hot = og_df.copy()
df_one_hot.drop(columns="class", inplace=True)
df_one_hot[cont_cols] = MinMaxScaler().fit_transform(df_one_hot[cont_cols])
for col in cat_cols:
    df_one_hot = encode_feature(df_one_hot, col)
df_one_hot

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,0.301370,0.044131,0.800000,0.021740,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.452055,0.048052,0.800000,0.000000,0.0,0.122449,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.287671,0.137581,0.533333,0.000000,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.493151,0.150486,0.400000,0.000000,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.150685,0.220635,0.800000,0.000000,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0.301370,0.137428,0.800000,0.000000,0.0,0.357143,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
48838,0.643836,0.209130,0.533333,0.000000,0.0,0.397959,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
48839,0.287671,0.245379,0.800000,0.000000,0.0,0.500000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
48840,0.369863,0.048444,0.800000,0.054551,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [12]:
kmeans = KMeans(n_clusters=2, n_init="auto", random_state=0).fit(df_one_hot)
kmeans_acc = cluster_accuracy(kmeans.labels_, og_df["class"].to_numpy())
kmeans_acc

0.7165758977928832

In [14]:
no_target_df = og_df.drop(columns="class")
distance_matrix = gower_matrix_duped(no_target_df)
distance_matrix

100%|██████████| 48842/48842 [19:47<00:00, 41.14it/s]  


array([[0.        , 0.3179897 , 0.31396827, ..., 0.30990908, 0.2932583 ,
        0.31081456],
       [0.3179897 , 0.        , 0.41400644, ..., 0.19566154, 0.38661748,
        0.12513845],
       [0.31396827, 0.41400644, 0.        , ..., 0.31975034, 0.32089615,
        0.3953219 ],
       ...,
       [0.30990908, 0.19566154, 0.31975034, ..., 0.        , 0.31683698,
        0.16239977],
       [0.2932583 , 0.38661748, 0.32089615, ..., 0.31683698, 0.        ,
        0.38917103],
       [0.31081456, 0.12513845, 0.3953219 , ..., 0.16239977, 0.38917103,
        0.        ]], dtype=float32)

In [22]:
gower_agglo = AgglomerativeClustering(n_clusters=2, metric="precomputed", linkage="single").fit_predict(distance_matrix)
gower_agglo_acc = cluster_accuracy(gower_agglo, og_df["class"].to_numpy())
gower_agglo_acc
# linkage=average: 0.7602882764833545
# linkage=single: 0.760697760124483

0.760697760124483

In [10]:
embedding_sizes = [(df[col].nunique(), min(50, max(2, (df[col].nunique()+1) // 2))) for col in df[cat_cols]]
embedding_sizes

[(9, 5), (16, 8), (7, 4), (15, 8), (6, 3), (5, 3), (2, 2), (42, 21)]

In [11]:
class CensusIncomeDataset(Dataset):
    def __init__(self, df):
        self.cat = torch.tensor(df[cat_cols].values, dtype=torch.float)
        self.cont = torch.tensor(df[cont_cols].values, dtype=torch.float)

    def __getitem__(self, idx):
        return self.cat[idx], self.cont[idx]

    def __len__(self):
        return self.cat.shape[0]
    
dataset = CensusIncomeDataset(df)
dataloader = DataLoader(dataset, batch_size=512, shuffle=True)
len(dataset)

48842